In [ ]:
from torchvision import datasets

import numpy as np

# preprocessing
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split

# model
import torch
import torchvision.models as models
from torch import nn
from torch.optim import Adam

# 1. Load Data

In [ ]:
def load_data(data_dir):
    """ Create train and test pytorch dataset objects from CIFAR10.
    
    The following tranformations are applied on CIFAR10:
        * resize images,
        * center crop images,
        * image pixels to [0, 1] range,
        * normalization (by substracting mean and dividing with std according to ImageNet)
    
    Args:
        data_dir:
            directory where data will be saved, as a string.
    
    Returns:
        train and test dataset, as pytorch dataset objects.
    """
    transform = transforms.Compose([
        transforms.Resize(224), 
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    trainset = datasets.CIFAR10(root=data_dir,
                                train=True, 
                                download=True, 
                                transform=transform)

    testset = datasets.CIFAR10(root=data_dir, 
                               train=False, 
                               download=True, 
                               transform=transform)

    return trainset, testset

In [ ]:
training_data, test_data = load_data(data_dir='cifar10')

print(f'\nTraining data:\n--------------\n{training_data}')
print(f'Test data:\n--------------\n{test_data}')

# 2. AlexNet

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    """ This helper function sets the .requires_grad attribute of the parameters in the model 
    to False when we are feature extracting. 
    
    When we are feature extracting and only want to compute gradients for the newly initialized layer 
    then we want all of the other parameters to not require gradients.
    
    Args:
        model: 
            deep learning model, as pytorch object.
        feature_extracting:
            whether or not we're feature extracting, as boolean.
    """
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(num_labels, feature_extract, pretrained=True):
    """ Initialize AlexNet model and reshape the last layer with the correct number of classes.
    
    Since AlexNet has been pretrained on Imagenet, it has output layers of size 1000, one node for each class. 
    We reshape the last layer to have the same number of inputs as before, and to have the same number of 
    outputs as the number of classes in our the dataset.
    
    Args:
        num_labels:
            number of labels in our dataset, as integer.
        feature_extract:
          flag for feature extracting (when False, we finetune the whole model, 
          when True we only update the reshaped layer params), as boolean.
        pretrained:
            whether or not we want the pretrained version of AlexNet, as boolean.
    
    Returns:
        AlexNet model, as pytorch object
    """
    model = models.alexnet(pretrained=pretrained)
    
    set_parameter_requires_grad(model, feature_extract)
    
    last_layer_in_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(last_layer_in_ftrs, num_labels)
    
    return model

# 3. Fit and Predict

In [ ]:
def fit(dataloader, model, loss_fn, optimizer, print_loss=False):
    """ Fit neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
        optimizer:
            optimizer function, as pytorch object.
        print_loss:
            print loss on every batch, as boolean (default False)
    """
    size = len(dataloader.dataset)
    model.train()  # put on train mode
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device), Y.to(device)
        
        # compute prediction
        pred = model(X)
        
        # compute loss
        loss = loss_fn(pred, Y)

        # reset the gradients
        optimizer.zero_grad()
        
        # backpropagate
        loss.backward()
        
        # update parameters
        optimizer.step()

        if print_loss and batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def predict(dataloader, model, loss_fn):
    """ Predict with neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
    """
    size = len(dataloader.dataset)
    
    test_loss = 0
    correct = 0
    
    model.eval()  # put on evaluation mode
    with torch.no_grad():
        for X, Y in dataloader:
            X, Y = X.to(device), Y.to(device)
            
            pred = model(X)
            
            test_loss += loss_fn(pred, Y).item()
            
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"average loss: {test_loss:>8f} \naccuracy: {(100*correct):>0.1f}%\n")

# 4. Final Model

In [ ]:
batch_size = 64

training_data, test_data = load_data(data_dir='cifar10')

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
num_labels=10
feature_extract=True
pretrained=True

model = initialize_model(num_labels=num_labels, feature_extract=feature_extract, pretrained=pretrained)
model.to(device)

print(f'Model architecture:\n{model}')

Create an optimizer that **only** updates the desired parameters

In [ ]:
params_to_update = model.parameters()
print("Parameters to learn:")
if feature_extract:
    params_to_update = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t", name)
else:
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            print("\t", name)

learning_rate = 0.001
optimizer = Adam(params_to_update, lr=learning_rate)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    fit(train_dataloader, model, loss_fn, optimizer, print_loss=False)
    
    print('\nTrain:\n-------')
    predict(train_dataloader, model, loss_fn)
    
    print('\nTest:\n-------')
    predict(test_dataloader, model, loss_fn)
    
    print(f"-------------------------------")